In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
import xgboost as xgb
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, roc_curve, auc,
                             classification_report)

In [2]:
df_train=pd.read_csv('ready_train.csv')
df_test=pd.read_csv('ready_validation.csv')

In [6]:
X_train = df_train.drop('Attrition_Flag', axis=1)
y_train = df_train['Attrition_Flag']
X_test = df_test.copy()

In [7]:
req_cols = ['Customer_Age', 'Dependent_count', 'Months_on_book', 'Total_Relationship_Count',
            'Months_Inactive_12_mon', 'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
            'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1',
            'Avg_Utilization_Ratio']

In [8]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train[req_cols])
X_test = scaler.transform(X_test[req_cols])

In [9]:
sm = SMOTE(random_state = 17, sampling_strategy = 1.0)
X_train, y_train = sm.fit_resample(X_train, y_train)

In [10]:
xgb_model = xgb.XGBClassifier(random_state=17, use_label_encoder=False, n_jobs=-1)
xgb_model.fit(X_train, y_train)

C:\Anaconda\lib\site-packages\xgboost\sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=-1, num_parallel_tree=1, predictor='auto', random_state=17, ...)

In [13]:
predictions=xgb_model.predict(X_test)

In [21]:
df_result=pd.DataFrame({'client_id':df_test['CLIENTNUM'],'churn':predictions})
df_result.to_csv('7_result_mka.csv',index=False)